Chain without chat history

In [ ]:
from auto_flow.core.llm.openai.openai_llm import OpenAILLM

llm = OpenAILLM(model="gpt-4o", temperature=0)

In [ ]:
from auto_flow.core.rag.splitter.text_splitter import TextSplitter
from auto_flow.core.rag.vectorstore.chroma import Chroma
from auto_flow.core.rag.document_loaders.dir_loader import DirLoader

docs = DirLoader(file_or_dir="./files/paul_graham_essay.txt").load()
docs = TextSplitter().split_document(docs)
vectorstore = Chroma()
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever(top_k=10)

In [ ]:
from auto_flow.core.rag.document.document import Document
from typing import List


def format_docs(documents: List[Document]):
    return "\n\n".join(doc.text for doc in documents)

In [ ]:
from auto_flow.core.llm.message_parser import StrOutParser
from auto_flow.core.rag.prompts import qa_prompt
from auto_flow.core.flow.flow import identity

rag_flow = (
        {"context": retriever | format_docs, "input": identity}
        | qa_prompt
        | llm
        | StrOutParser()
)

In [ ]:
rag_flow.invoke("what did paul graham do after going to RISD?")

Implement with helpers

In [ ]:
from auto_flow.core.rag.helpers import create_qa_flow

rag_flow2 = create_qa_flow(retriever, llm)
rag_flow2.invoke("what did paul graham do after going to RISD?")

Contextualizing the question

In [ ]:
from auto_flow.core.rag.retrivers.history_aware_retriever import create_history_aware_retriever

history_aware_retriever = create_history_aware_retriever(llm, retriever)

Chain with chat history

In [ ]:

from auto_flow.core.rag.helpers import create_rag_flow
from auto_flow.core.rag.combine_documents.stuff import create_stuff_documents_flow
from auto_flow.core.rag.prompts import qa_prompt

question_answer_flow = create_stuff_documents_flow(qa_prompt, llm)
rag_flow = create_rag_flow(history_aware_retriever, question_answer_flow)

In [ ]:
from auto_flow.core.messages.chat_message import ChatMessage, Role

chat_history = []

question = "what did paul graham do after going to RISD?"
ai_msg_1 = rag_flow.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([ChatMessage(role=Role.USER, content=question), ai_msg_1["answer"]])

second_question = "Where dit he work after that?"
ai_msg_2 = rag_flow.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Implement with helpers

In [ ]:
from auto_flow.core.rag.helpers import create_qa_flow_with_history

chat_history = []

rag_flow2 = create_qa_flow_with_history(retriever, llm)
question = "what did paul graham do after going to RISD?"
ai_msg_1 = rag_flow2.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([ChatMessage(role=Role.USER, content=question), ai_msg_1])

second_question = "Where dit he work after that?"
ai_msg_2 = rag_flow2.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2)